# MDF data analysis

In [1]:
from load_data import train, test, BIG, TRAIN, TEST, SOURCE
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

TRAIN (8564, 42)
TEST  (3671, 42)
BIG  (12235, 43)
height has been deprecated.



In [2]:
from visualization_helper import display_groups, display_unique_count
from preprocessing_helper import get_unique_noconst, split_series_values, get_unique_col_values, get_numeric_nonnumeric_cols

In [3]:
print BIG.shape, BIG.columns.values
print train.shape, train.columns.values
print test.shape, test.columns.values


(12235, 43) [u'agrement col' u'date amm annee' u'date declar annee' u'etat commerc'
 u'forme pharma' u'id' u'libelle' u'libelle_ampoule' u'libelle_capsule'
 u'libelle_comprime' u'libelle_film' u'libelle_flacon' u'libelle_gelule'
 u'libelle_pilulier' u'libelle_plaquette' u'libelle_poche'
 u'libelle_sachet' u'libelle_seringue' u'libelle_stylo' u'libelle_tube'
 u'nb_ampoule' u'nb_capsule' u'nb_comprime' u'nb_film' u'nb_flacon'
 u'nb_gelule' u'nb_ml' u'nb_pilulier' u'nb_plaquette' u'nb_poche'
 u'nb_sachet' u'nb_seringue' u'nb_stylo' u'nb_tube' u'prix' 'source'
 u'statut' u'statut admin' u'substances' u'titulaires' u'tx rembours'
 u'type proc' u'voies admin']
(8564, 42) [u'libelle' u'libelle_plaquette' u'libelle_ampoule' u'libelle_flacon'
 u'libelle_tube' u'libelle_stylo' u'libelle_seringue' u'libelle_pilulier'
 u'libelle_sachet' u'libelle_comprime' u'libelle_gelule' u'libelle_film'
 u'libelle_poche' u'libelle_capsule' u'nb_plaquette' u'nb_ampoule'
 u'nb_flacon' u'nb_tube' u'nb_stylo' u'nb_

Remove some useless columns

In [4]:
df__noid_noprix = BIG.drop(['libelle', 'prix', 'id'], axis=1)

Check for dupicate rows and lets look at the columns with only one unique value.

In [5]:
df__noid_noprix_unique_noconst = get_unique_noconst(df__noid_noprix, verbose=True)

-- get_unique_noconst --
- Number of found duplicated rows :  1389
- Remove constant columns :  (10846, 40) -> (10846, 40)
- Dropped const columns : 


Columns description : 

In [6]:
type_groups_func = lambda df : df.columns.to_series().groupby(df.dtypes)
type_groups = type_groups_func(df__noid_noprix_unique_noconst)

In [7]:
display_groups(type_groups)


-- object --

	agrement col
	etat commerc
	forme pharma
	source
	statut
	statut admin
	substances
	titulaires
	tx rembours
	type proc
	voies admin

-- int64 --

	date amm annee
	date declar annee
	libelle_ampoule
	libelle_capsule
	libelle_comprime
	libelle_film
	libelle_flacon
	libelle_gelule
	libelle_pilulier
	libelle_plaquette
	libelle_poche
	libelle_sachet
	libelle_seringue
	libelle_stylo
	libelle_tube
	nb_ampoule
	nb_capsule
	nb_comprime
	nb_film
	nb_flacon
	nb_gelule
	nb_pilulier
	nb_plaquette
	nb_poche
	nb_sachet
	nb_seringue
	nb_stylo
	nb_tube

-- float64 --

	nb_ml


Identify and separate the numeric and non numeric rows.
Report NaN values on numerical columns

In [8]:
num_cols, nonnum_cols = get_numeric_nonnumeric_cols(df__noid_noprix_unique_noconst)
        
nb_isnan = np.sum(df__noid_noprix_unique_noconst[num_cols].apply(np.isnan))

In [9]:
print nb_isnan

date amm annee       0
date declar annee    0
libelle_ampoule      0
libelle_capsule      0
libelle_comprime     0
libelle_film         0
libelle_flacon       0
libelle_gelule       0
libelle_pilulier     0
libelle_plaquette    0
libelle_poche        0
libelle_sachet       0
libelle_seringue     0
libelle_stylo        0
libelle_tube         0
nb_ampoule           0
nb_capsule           0
nb_comprime          0
nb_film              0
nb_flacon            0
nb_gelule            0
nb_pilulier          0
nb_plaquette         0
nb_poche             0
nb_sachet            0
nb_seringue          0
nb_stylo             0
nb_tube              0
nb_ml                0
dtype: int64


Identify non numeric rows.

In [10]:
df__noid_noprix_unique_noconst[nonnum_cols].tail(5)

,agrement col,etat commerc,forme pharma,source,statut,statut admin,substances,titulaires,tx rembours,type proc,voies admin
12230,oui,Déclaration de commercialisation,solution injectable,test,Présentation active,Autorisation active,KÉTOPROFÈNE,PHARMY II,30%,Procédure nationale,intramusculaire
12231,oui,Déclaration de commercialisation,solution injectable,test,Présentation active,Autorisation active,ACIDE GADOTÉRIQUE,GUERBET,65%,Procédure nationale,intra-articulaire
12232,oui,Déclaration de commercialisation,comprimé gastro-résistant(e),test,Présentation active,Autorisation active,"PANTOPRAZOLE SODIQUE SESQUIHYDRATÉ, PANTOPRAZOLE",RANBAXY PHARMACIE GENERIQUES,65%,Procédure décentralisée,orale
12233,oui,Déclaration de commercialisation,solution pour perfusion,test,Présentation active,Autorisation active,"ACIDE ZOLÉDRONIQUE ANHYDRE, ACIDE ZOLÉDRONIQUE...",BIOGARAN,65%,Procédure nationale,intraveineuse
12234,oui,Déclaration de commercialisation,capsule molle,test,Présentation active,Autorisation active,"CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",PHARMA DEVELOPPEMENT,65%,Procédure nationale,orale


In [11]:
train__noid_noprix_unique_noconst = df__noid_noprix_unique_noconst[df__noid_noprix_unique_noconst[SOURCE] == TRAIN]
train_unique_col_values = get_unique_col_values(train__noid_noprix_unique_noconst[nonnum_cols])
train_unique_col_values

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537,1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610,1611,1612,1613,1614,1615,1616,1617,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693,1694,1695,1696,1697,1698,1699,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712,1713,1714,1715,1716,1717,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741
0,agrement col,2,oui,non,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,etat commerc,4,Déclaration de commercialisation,Déclaration d'arrêt de commercialisation,Arrêt de commercialisation (le médicament n'a ...,Déclaration de suspension de commercialisation,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,forme pharma,190,comprimé pelliculé,comprimé à croquer,gélule à libération prolongée,granulés à libération prolongée,comprimé,capsule molle,comprimé sécable,dispositif,poudre pour inhalation,solution buvable gouttes,comprimé dispersible orodispersible,comprimé pelliculé sécable,comprimé quadrisécable,comprimé pelliculé à libération prolongée,gélule gastro-résistant(e),gélule,collyre en solution,poudre et solvant pour solution injectable,solution gouttes,comprimé orodispersible,solution injectable,émulsion fluide pour application,comprimé effervescent(e) sécable,suppositoire,poudre pour solution pour irrigation vésical(e),poudre pour s

In [12]:
test__noid_noprix_unique_noconst = df__noid_noprix_unique_noconst[df__noid_noprix_unique_noconst[SOURCE] == TEST]
test_unique_col_values = get_unique_col_values(test__noid_noprix_unique_noconst[nonnum_cols])
test_unique_col_values

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128,1129,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140,1141,1142,1143
0,agrement col,2,non,oui,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,etat commerc,3,Déclaration d'arrêt de commercialisation,Déclaration de commercialisation,Arrêt de commercialisation (le médicament n'a ...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,forme pharma,159,comprimé gastro-résistant(e),capsule,comprimé,suspension injectable,émulsion pour perfusion,comprimé pelliculé sécable,solution pour perfusion,comprimé pelliculé,solution injectable,comprimé sécable,émulsion pour application,collyre,gélule gastro-résistant(e),gélule,comprimé effervescent(e) sécable,poudre pour suspension buvable,collyre en solution,solution buvable,solution et solution pour perfusion,comprimé dispersible sécable,solution pour inhalation,comprimé effervescent(e),gélule à libération prolongée,solution buvable en gouttes,comprimé à libération prolongée,granulés effervescent(e) pour solution buvable,comprimé orodispersible,dispositif,poudre et solvant pour suspension injectable,suspension buvable,solution pour application,comprimé à libération modifiée,c

In [ ]:
all_unique_col_values = get_unique_col_values(df__noid_noprix_unique_noconst[nonnum_cols])
all_unique_col_values

In [13]:
index = 1
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

Empty DataFrame
Columns: [Value, Count]
Index: []


In [14]:
index = 2
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

                                                 Value  Count
113                              pâte pour application      1
84   microsphère et solution pour usage parentéral ...      1
97      poudre et solvant pour solution pour perfusion      1
145                        suspension buvable à diluer      1
148          suspension par nébuliseur pour inhalation      1
151                       suspension pour instillation      1
153                               système de diffusion      1
156                 émulsion injectable pour perfusion      1
121                                solution buvable ou      1
126                solution filmogène pour application      1
132                 solution pour application à diluer      1
79                   lyophilisat pour usage parentéral      1
24                      comprimé et comprimé pelliculé      1
26                               comprimé muco-adhésif      1
28                     comprimé orodispersible sécable      1
19      

In [15]:
index = 4
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

Empty DataFrame
Columns: [Value, Count]
Index: []


In [16]:
index = 5
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

                    Value  Count
4  Autorisation suspendue      1


In [47]:
index = 6
ll = train_unique_col_values.iloc[index][1]

def remove_commas_in_brackets(s):
    import re
    groups = re.findall(r'\(.*?\)', s)
#    print groups, len(groups)
    if len(groups) > 0:
        for g in groups:
#            print "Group:", g
            repl = g.replace(',', ' ')
            return s.replace(unicode(g), unicode(repl))
    return s

series = train_unique_col_values.iloc[index][2:2+ll]
#series = series.apply(remove_commas_in_brackets)

series[series.str.contains("A/SWITZERLAND/9715293/2013")].index
#print series[3]
print series[1185]

# train_res = split_series_values(series)    
# train_res[train_res.str.contains("A/SWITZERLAND/9715293/2013")].index
# train_res[2520]
# train_res[2521]

#train_df = display_unique_count(train_res)



# ll = test_unique_col_values.iloc[index][1]
# test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
# test_df = display_unique_count(test_res)

# print test_df[~test_df['Value'].isin(train_df['Value'])]

VIRUS DE LA GRIPPE FRAGMENTÉ, INACTIVÉ, SOUCHE B/PHUKET/3073/2013 , VIRUS DE LA GRIPPE FRAGMENTÉ, INACTIVÉ, SOUCHE A/CALIFORNIA/7/2009 (H1N1) PDM09 - SOUCHE ANALOGUE (A/CHRISTCHURCH/16/2010, NIB-74XP , VIRUS DE LA GRIPPE FRAGMENTÉ, INACTIVÉ, SOUCHE A/SWITZERLAND/9715293/2013 (H3N2) - SOUCHE ANALOGUE (A/SWITZERLAND/9715293/2013, NIB-88) 


In [46]:
test = u'VIRUS DE LA GRIPPE FRAGMENT\xc9, INACTIV\xc9, SOUCHE B/PHUKET/3073/2013 , VIRUS DE LA GRIPPE FRAGMENT\xc9, INACTIV\xc9, SOUCHE A/CALIFORNIA/7/2009 (H1N1) PDM09 - SOUCHE ANALOGUE (A/CHRISTCHURCH/16/2010, NIB-74XP , VIRUS DE LA GRIPPE FRAGMENT\xc9, INACTIV\xc9, SOUCHE A/SWITZERLAND/9715293/2013 (H3N2) - SOUCHE ANALOGUE (A/SWITZERLAND/9715293/2013, NIB-88) '

print remove_commas_in_brackets(test)


[u'(H1N1)', u'(A/CHRISTCHURCH/16/2010, NIB-74XP , VIRUS DE LA GRIPPE FRAGMENT\xc9, INACTIV\xc9, SOUCHE A/SWITZERLAND/9715293/2013 (H3N2)', u'(A/SWITZERLAND/9715293/2013, NIB-88)'] 3
Group: (H1N1)
VIRUS DE LA GRIPPE FRAGMENTÉ, INACTIVÉ, SOUCHE B/PHUKET/3073/2013 , VIRUS DE LA GRIPPE FRAGMENTÉ, INACTIVÉ, SOUCHE A/CALIFORNIA/7/2009 (H1N1) PDM09 - SOUCHE ANALOGUE (A/CHRISTCHURCH/16/2010, NIB-74XP , VIRUS DE LA GRIPPE FRAGMENTÉ, INACTIVÉ, SOUCHE A/SWITZERLAND/9715293/2013 (H3N2) - SOUCHE ANALOGUE (A/SWITZERLAND/9715293/2013, NIB-88) 


In [ ]:
index = 7
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

In [ ]:
index = 8
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

In [ ]:
index = 9
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

In [ ]:
index = 10
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

Encode categorical features

In [ ]:
df = df__noid_noprix_unique_noconst[nonnum_cols].drop(['substances', 'voies admin'], axis=1)
out_df = pd.DataFrame(columns=df.columns, index=df.index)
for c in df.columns:
    le = LabelEncoder()    
    le.fit(df[c])
    out_df[c] = le.transform(df[c])
out_df

In [ ]:
import re
a0="(ASWITZERLAND51351 , NIB-88)"
a="(A/SWITZERLAND/9715293/201, dsdf, NIB-88)"
a11="(A/SWITZERLAND/9715293/2013, NIB-88)"
a1="- SOUCHE ANALOGUE (A/SWITZERLAND/9715293/2013, NIB-88) , ANTIGÈNES DE SURFACE DU VIRUS DE LA GRIPPE, "
a2="SOUCHE A/CALIFORNIA/7/2009 (H1N1)PDM09 - SOUCHE ANALOGUE (A/CALIFORNIA/7/2009, X-181)"
a3="SOUCHE A/CALIFORNIA/7/2009 (H1N1)PDM09 - SOUCHE ANALOGUE (A/CALIFORNIA/7/2009, X-181), - SOUCHE ANALOGUE (A/CALIFORNIA/sadsa7/2009, asd, X-181),"
b="sdfsdfsdfdf"

s = a3
groups = re.findall(r'\(.*?\)', s)
if len(groups) > 0:
    for g in groups:
        print g
        repl = g.replace(',', ' ')
        s = s.replace(g, repl)
    print s
else:
    print "Not found"
    
# b = re.sub(r'\(.*[,].*\)', r' ', a)
# print b